##### Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Abstract

Reference implementation for [ITE-5: Signature scheme that avoids canonicalization](https://github.com/MarkLodato/ITE/tree/ite-5/ITE/5).

_Author: Mark Lodato, Google, <lodato@google.com>_  
_Date: September 2020_

(To edit, [open this doc in Colab](https://colab.research.google.com/github/MarkLodato/ITE/blob/ite-5/ITE/5/reference_implementation.ipynb).)

## Implementation

In [ ]:
import base64, binascii, json, struct

def b64enc(m: bytes) -> str:
  return base64.standard_b64encode(m).decode('utf-8')

def b64dec(m: str) -> bytes:
  m = m.encode('utf-8')
  try:
    return base64.b64decode(m, validate=True)
  except binascii.Error:
    return base64.b64decode(m, altchars='-_', validate=True)

def PAE(payloadType: str, payload: bytes) -> bytes:
  return b''.join([struct.pack('<Q', 2),
                   struct.pack('<Q', len(payloadType)),
                   payloadType.encode('utf-8'),
                   struct.pack('<Q', len(payload)),
                   payload])

def Sign(payloadType: str, payload: bytes, signer) -> str:
  return json.dumps({
      'payload': b64enc(payload),
      'payloadType': payloadType,
      'signatures': [{"sig": b64enc(signer.sign(PAE(payloadType, payload)))}],
  })

def Verify(json_signature: str, verifier) -> (str, bytes):
  wrapper = json.loads(json_signature)
  payloadType = wrapper['payloadType']
  payload = b64dec(wrapper['payload'])
  pae = PAE(payloadType, payload)
  for signature in wrapper['signatures']:
    if verifier.verify(pae, b64dec(signature['sig'])):
      break
  else:
    raise ValueError('No valid signature found')
  return payloadType, payload

## Dummy crypto implementation

In [ ]:
!pip install pycryptodome

In [ ]:
from Crypto.Hash import SHA256
from Crypto.PublicKey import ECC
from Crypto.Signature import DSS

class Signer:
  def __init__(self, secret_key):
    self.secret_key = secret_key
    self.public_key = self.secret_key.public_key()

  @classmethod
  def generate(cls):
    return cls(ECC.generate(curve='P-256'))

  def sign(self, message: bytes) -> bytes:
    """Returns the signature of `message`."""
    h = SHA256.new(message)
    return DSS.new(self.secret_key, 'deterministic-rfc6979').sign(h)


class Verifier:
  def __init__(self, public_key):
    self.public_key = public_key

  def verify(self, message: bytes, signature: bytes) -> bool:
    """Returns true if `message` was signed by `signature`."""
    h = SHA256.new(message)
    try:
      DSS.new(self.public_key, 'fips-186-3').verify(h, signature)
      return True
    except ValueError:
      return False

## Example

In [ ]:
signer = Signer.generate()
verifier = Verifier(signer.public_key)
print('Algorithm: ECDSA with deterministic-rfc6979 and SHA256')
print('Curve:', signer.secret_key.curve)
print('Public X:', signer.secret_key.pointQ.x)
print('Public Y:', signer.secret_key.pointQ.y)
print('Private d:', signer.secret_key.d)

Algorithm: ECDSA with deterministic-rfc6979 and SHA256
Curve: NIST P-256
Public X: 46950820868899156662930047687818585632848591499744589407958293238635476079160
Public Y: 5640078356564379163099075877009565129882514886557779369047442380624545832820
Private d: 97358161215184420915383655311931858321456579547487070936769975997791359926199


In [ ]:
signature_json = Sign('http://example.com/HelloWorld', b'hello world', signer)
json.loads(signature_json)

{'payload': 'aGVsbG8gd29ybGQ=',
 'payloadType': 'http://example.com/HelloWorld',
 'signatures': [{'sig': 'y7BK8Mm8Mr4gxk4+G9X3BD1iBc/vVVuJuV4ubmsEK4m/8MhQOOS26ejx+weIjyAx8VjYoZRPpoXSNjHEzdE7nQ=='}]}

In [ ]:
Verify(signature_json, verifier)

('http://example.com/HelloWorld', b'hello world')

In [ ]:
import binascii, textwrap
def print_hex(b: bytes):
  octets = ' '.join(textwrap.wrap(binascii.hexlify(b).decode('utf-8'), 2))
  print(*textwrap.wrap(octets, 48), sep='\n')

print_hex(PAE('http://example.com/HelloWorld', b'hello world'))

02 00 00 00 00 00 00 00 1d 00 00 00 00 00 00 00
68 74 74 70 3a 2f 2f 65 78 61 6d 70 6c 65 2e 63
6f 6d 2f 48 65 6c 6c 6f 57 6f 72 6c 64 0b 00 00
00 00 00 00 00 68 65 6c 6c 6f 20 77 6f 72 6c 64
